## Text to SQL generation using Code-Bison

#### Imports 

In [1]:
from vertexai.language_models import CodeGenerationModel
from google.cloud import bigquery
import logging 
import os 

##### Setup logging

In [2]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

#### Setup essentials

In [3]:
SERVICE_ACCOUNT_CREDENTIALS = './../credentials/vai-key.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = SERVICE_ACCOUNT_CREDENTIALS

In [4]:
PROJECT_ID = 'arun-genai-bb'
CODE_GEN_MODEL_NAME = 'code-bison@latest'
TEMPERATURE = 0.8 # default value = 0
MAX_OUTPUT_TOKENS = 2048  # length of the output response | overridding the default value which is 128
# TOP_P = 0.95  # default value
# TOP_K = 40  # default value
LOCATION = 'us-central1'

In [5]:
DATASET = 'flight_reservations'
TABLES = ['customers', 'flights', 'reservations', 'transactions', 'loyality_points']

In [6]:
code_gen_model = CodeGenerationModel.from_pretrained(CODE_GEN_MODEL_NAME)
bq_client = bigquery.Client()

In [7]:
query = f"""
    SELECT *
    FROM `{PROJECT_ID}.{DATASET}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
    WHERE table_name in ({','.join([f'"{table}"' for table in TABLES])})
"""
logger.info(query)


    SELECT *
    FROM `arun-genai-bb.flight_reservations.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
    WHERE table_name in ("customers","flights","reservations","transactions","loyality_points")



In [8]:
schema_columns = bq_client.query(query=query).to_dataframe()
schema_columns

,table_catalog,table_schema,table_name,column_name,field_path,data_type,description,collation_name,rounding_mode
0,arun-genai-bb,flight_reservations,transactions,transaction_id,transaction_id,INT64,None,NULL,None
1,arun-genai-bb,flight_reservations,transactions,reservation_id,reservation_id,INT64,None,NULL,None
2,arun-genai-bb,flight_reservations,transactions,amount,amount,FLOAT64,None,NULL,None
3,arun-genai-bb,flight_reservations,transactions,transaction_datetime,transaction_datetime,DATETIME,None,NULL,None
4,arun-genai-bb,flight_reservations,reservations,reservation_id,reservation_id,INT64,None,NULL,None
5,arun-genai-bb,flight_reservations,reservations,customer_id,customer_id,INT64,None,NULL,None
6,arun-genai-bb,flight_reservations,reservations,flight_id,flight_id,INT64,None,NULL,None
7,arun-genai-bb,flight_reservations,reservations,reservation_datetime,reservation_datetime,DATETIME,None,NULL,None
8,arun-genai-bb,flight_reservations,reservations,status,status,STRING,None,NULL,None
9,arun-genai-bb,flight_reservations,flights,flight_id,flight_id,INT64,None,NULL,None


In [9]:
schema_columns = schema_columns.to_markdown(index=False)
logger.info(schema_columns)

| table_catalog   | table_schema        | table_name   | column_name          | field_path           | data_type   | description   | collation_name   | rounding_mode   |
|:----------------|:--------------------|:-------------|:---------------------|:---------------------|:------------|:--------------|:-----------------|:----------------|
| arun-genai-bb   | flight_reservations | transactions | transaction_id       | transaction_id       | INT64       |               | NULL             |                 |
| arun-genai-bb   | flight_reservations | transactions | reservation_id       | reservation_id       | INT64       |               | NULL             |                 |
| arun-genai-bb   | flight_reservations | transactions | amount               | amount               | FLOAT64     |               | NULL             |                 |
| arun-genai-bb   | flight_reservations | transactions | transaction_datetime | transaction_datetime | DATETIME    |               | NULL             

Utility function to convert text into SQL and automatically correct the SQL query if any execution errors occur. <br>
The function also collects the successful query executions, ranks them by latency of execution, and returns the fastest. <br>
Additionally, it provides an option to retrieve all the successful queries and their latencies as a dataframe

In [10]:
import pandas as pd
import time

def generate_and_execute_sql(prompt, max_tries=5, return_all=False):
    """
    Generate an SQL query using the code_gen_model, execute it using bq_client, and rank successful queries by latency.
    
    Args:
    - prompt (str): Prompt to provide to the model for generating SQL.
    - max_tries (int): Maximum number of attempts to generate and execute SQL.
    - return_all (bool): Flag to determine whether to return all successful queries or only the fastest.
    
    Returns:
    - dict: A dictionary containing the fastest dataframe or all successful dataframes, or error messages and prompt evolution.
    """
    
    tries = 0
    error_messages = []
    prompts = [prompt]
    successful_queries = []
    
    while tries < max_tries:
        logger.info(f'TRIAL: {tries+1}')
        try:
            # Predict SQL using the model
            start_time = time.time()
            response = code_gen_model.predict(prompt, temperature=TEMPERATURE, max_output_tokens=MAX_OUTPUT_TOKENS)
            generated_sql_query = response.text
            generated_sql_query = '\n'.join(generated_sql_query.split('\n')[1:-1])
            logger.info('-' * 50)
            logger.info(generated_sql_query)
            logger.info('-' * 50)
            # Execute SQL using BigQuery client
            df = bq_client.query(generated_sql_query).to_dataframe()
            latency = time.time() - start_time
            successful_queries.append({
                "query": generated_sql_query,
                "dataframe": df,
                "latency": latency
            })
            logger.info('SUCCEEDED')
        except Exception as e:
            logger.error('FAILED')
            # Catch the error, store the message, and try again
            msg = str(e)
            error_messages.append(msg)
            # Evolve the prompt by appending the error message and asking the model to correct it
            prompt = f"""{prompt}
Encountered an error: {msg}. 
To address this, please generate an alternative SQL query response that avoids this specific error. 
Follow the instructions mentioned above to remediate the error. 

Modify the below SQL query to resolve the issue:
{generated_sql_query}

Provide a detailed explanation of the modifications made and the rationale behind those changes. 
Ensure the revised SQL query aligns precisely with the requirements outlined in the initial question."""
            prompts.append(prompt)
        logger.info('=' * 100)
        tries += 1
    # If no successful queries
    if len(successful_queries) == 0:
        return {
            "error": "All attempts exhausted.",
            "prompts": prompts,
            "errors": error_messages
        }
    
    # Sort successful queries by latency
    successful_queries.sort(key=lambda x: x['latency'])
    
    if return_all:
        df = pd.DataFrame([(q["query"], q["latency"]) for q in successful_queries], columns=["Query", "Latency"])
        return {
            "dataframe": df
        }
    else:
        return {
            "fastest_query": successful_queries[0]["query"],
            "latency": successful_queries[0]["latency"]
        }

### Test text to SQL scenarios

Construct the SEED prompt

In [11]:
seed_prompt = """
Please craft a SQL query for BigQuery that addresses the following QUESTION provided below. 
Ensure you reference the appropriate BigQuery tables and column names provided in the SCHEMA below. 
When joining tables, employ type coercion to guarantee data type consistency for the join columns. 
Additionally, the output column names should specify units where applicable.\n
QUESTION:
{}\n
SCHEMA:
{}\n
IMPORTANT: 
Use ONLY DATETIME and DO NOT use TIMESTAMP.
--
Ensure your SQL query accurately defines both the start and end of the DATETIME range.
"""
logger.info(seed_prompt)


Please craft a SQL query for BigQuery that addresses the following QUESTION provided below. 
Ensure you reference the appropriate BigQuery tables and column names provided in the SCHEMA below. 
When joining tables, employ type coercion to guarantee data type consistency for the join columns. 
Additionally, the output column names should specify units where applicable.

QUESTION:
{}

SCHEMA:
{}

IMPORTANT: 
Use ONLY DATETIME and DO NOT use TIMESTAMP.
--
Ensure your SQL query accurately defines both the start and end of the DATETIME range.



#### Scenario 1: Retrieve Active Reservations for a Specific Date Range

For this scenario, you want to find all active reservations within a specific date range.

In [12]:
question = "Provide a list of all flight reservations from October 10th to October 15th, 2023"

In [13]:
prompt = seed_prompt.format(question, schema_columns)
logger.info(prompt)


Please craft a SQL query for BigQuery that addresses the following QUESTION provided below. 
Ensure you reference the appropriate BigQuery tables and column names provided in the SCHEMA below. 
When joining tables, employ type coercion to guarantee data type consistency for the join columns. 
Additionally, the output column names should specify units where applicable.

QUESTION:
Provide a list of all flight reservations from October 10th to October 15th, 2023

SCHEMA:
| table_catalog   | table_schema        | table_name   | column_name          | field_path           | data_type   | description   | collation_name   | rounding_mode   |
|:----------------|:--------------------|:-------------|:---------------------|:---------------------|:------------|:--------------|:-----------------|:----------------|
| arun-genai-bb   | flight_reservations | transactions | transaction_id       | transaction_id       | INT64       |               | NULL             |                 |
| arun-genai-bb 

In [14]:
%%time

response = generate_and_execute_sql(prompt=prompt, return_all=True)

TRIAL: 1
--------------------------------------------------
SELECT
  r.reservation_id,
  r.customer_id,
  r.flight_id,
  r.reservation_datetime,
  r.status,
  f.origin,
  f.destination,
  f.departure_datetime,
  f.arrival_datetime,
  f.carrier,
  f.price
FROM arun-genai-bb.flight_reservations.reservations r
JOIN arun-genai-bb.flight_reservations.flights f
ON CAST(r.flight_id AS STRING) = CAST(f.flight_id AS STRING)
WHERE r.reservation_datetime BETWEEN '2023-10-10 00:00:00' AND '2023-10-15 23:59:59'
ORDER BY r.reservation_datetime;
--------------------------------------------------
SUCCEEDED
TRIAL: 2
--------------------------------------------------
SELECT
  r.reservation_id,
  r.customer_id,
  r.flight_id,
  r.reservation_datetime,
  r.status,
  f.origin,
  f.destination,
  f.departure_datetime,
  f.arrival_datetime,
  f.carrier,
  f.price
FROM arun-genai-bb.flight_reservations.reservations r
JOIN arun-genai-bb.flight_reservations.flights f
ON CAST(r.flight_id AS STRING) = CAST(f.flig

CPU times: user 354 ms, sys: 62.6 ms, total: 417 ms
Wall time: 34.3 s


In [15]:
df = response['dataframe']
df


,Query,Latency
0,"SELECT\n r.reservation_id,\n r.customer_id,\...",5.544287
1,"SELECT\n r.reservation_id,\n r.customer_id,\...",6.390517
2,"SELECT\n r.reservation_id,\n r.customer_id,\...",6.495003
3,"SELECT\n r.reservation_id,\n r.customer_id,\...",7.808290
4,"SELECT\n r.reservation_id,\n r.customer_id,\...",8.050346


#### Scenario 2: Identify customers who made reservations in the past N days.

In [ ]:
question = "Identify all customers who have made flight reservations within the last 7 days."

In [ ]:
prompt = seed_prompt.format(question, schema_columns)

In [ ]:
%%time

response = generate_and_execute_sql(prompt=prompt)

In [ ]:
sql_output = response['dataframe']
sql_output

### Scenario 3: Calculate Monthly Revenue
Calculate the total revenue generated from transactions for a given month and year.

In [ ]:
question = "Calculate the total revenue generated from transactions in October 2023, specifically from all reservations with a Confirmed status."

In [ ]:
prompt = seed_prompt.format(question, schema_columns)

In [ ]:
%%time

response = generate_and_execute_sql(prompt=prompt)

In [ ]:
sql_output = response['dataframe']
sql_output

### Scenario 4: Popular Flight Times
Identify the most popular departure hours or days for a given day or month or year.

In [ ]:
question = "Determine the departure months with the highest frequency for the year 2023."

In [ ]:
prompt = seed_prompt.format(question, schema_columns)

In [ ]:
%%time

response = generate_and_execute_sql(prompt=prompt)

In [ ]:
sql_output = response['dataframe']
sql_output

### Scenario 5: Customer Age Group
Group customers by age brackets and count the number in each bracket.

In [ ]:
question = "Group customers into five distinct age brackets and count the number of customers in each bracket."

In [ ]:
prompt = seed_prompt.format(question, schema_columns)

In [ ]:
%%time

response = generate_and_execute_sql(prompt=prompt)

In [ ]:
sql_output = response['dataframe']
sql_output

### Scenario 6: Age Calculation
Calculate the age of customers based on their date of birth and filter those who are above X years old.

In [ ]:
question = "Identify and rank all customers aged 18 and over who have reservations with a `Confirmed` status for the current month, ordered by their age. Ensure to display their ages in the result."

In [ ]:
prompt = seed_prompt.format(question, schema_columns)

In [ ]:
%%time

response = generate_and_execute_sql(prompt=prompt)

In [ ]:
sql_output = response['dataframe']
sql_output

Inspect the evolution of the seed prompt and how SQL query was fixed automatically by the LLM.

In [ ]:
for i, prompt in enumerate(response['prompts']):
    logger.info(f'==================== ATTEMPT {i+1} ====================')
    logger.info(prompt)